In [2]:
import nltk
import pandas as pd
import numpy as np

import re
from functools import reduce
import openpyxl
import xlsxwriter

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer

import os
import importlib
import utilities
%matplotlib inline
from scipy.stats import pearsonr
import numpy as np

# load 

In [3]:
splist=['cbow_6_ukwac_subtitle','cbow_subtitle','glove_6B','TASA','ukwac']
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
resultFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/algos/'

### 1: 'cbow_6_ukwac_subtitle'
### 2: 'cbow_subtitle'
### 3: 'glove_6B'
### 4: 'TASA'

## Legend
### min: novelty measure between response and stim, semantic distance with "minimun similarity word"
### disp: dispersion of response set from same subject 
### orig: originality of a response based on freqency of occuring from all subjects
### volume: minimum volume of enclosing ellipsoid of the response set
### fluency: number of responses from a subject

## item code
### b_: box, r_: rope, br_:brick

In [191]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [192]:
def getPC(df,cols):
    ndata=(df[cols]-df[cols].mean())/df[cols].std()
    imp_mean.fit(ndata.values)
    pca = PCA(n_components=1)

    imputed=imp_mean.transform(ndata.values)
    pca.fit(imputed)
    pc1=(pca.fit_transform(imputed).T)[0]
  
    return (pc1-np.mean(pc1))/np.std(pc1)


# BRM study 1 - 3

In [153]:
df=pd.read_csv(resultFolder+'BRM_agg_meanmean.csv')
#df=pd.read_csv(resultFolder+'BRM_agg_maxmean.csv')
df.head()

,id,study,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,wordcnt,rater
0,1,2,0.935925,0.928005,0.926775,0.896130,2.250000,2.008450
1,1,3,0.877492,0.862785,0.862785,0.862785,1.000000,1.410000
2,2,2,0.929227,0.889161,0.825989,0.808062,1.433333,1.329606
3,2,3,1.005650,0.962950,0.928850,0.881350,3.500000,1.417500
4,3,2,0.980574,0.963390,0.940290,0.890422,3.298611,2.286986


In [154]:
studytag='s1'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
agg['study']=1
agg.columns


Index(['id', 'age', 'gender', 'b_rater1', 'b_rater2', 'b_rater3', 'b_rater4',
       'b_cbowu', 'b_cbows', 'b_cboww', 'b_tasa', 'b_glov', 'r_rater1',
       'r_rater2', 'r_rater3', 'r_rater4', 'r_cbowu', 'r_cbows', 'r_cboww',
       'r_tasa', 'r_glov', 'b_cbowu_a', 'b_cbows_a', 'b_cboww_a', 'b_tasa_a',
       'b_glov_a', 'r_cbowu_a', 'r_cbows_a', 'r_cboww_a', 'r_tasa_a',
       'r_glov_a', 'gf_cfiq', 'gf_letters', 'gf_numbers', 'bicb', 'caq_log',
       'icaa_act', 'icaa_ach', 'ssci_creative_identity',
       'ssci_creative_efficacy', 'n_ffi', 'e_ffi', 'o_ffi', 'a_ffi', 'c_ffi',
       'meta1_r1', 'meta1_r2', 'meta1_r3', 'meta1_r4', 'meta2_r1', 'meta2_r2',
       'meta2_r3', 'meta2_r4', 'study'],
      dtype='object')

In [155]:
studytag='s2'
file = studytag+'_data_agg.xlsx'
agg2=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
agg2['study']=2
agg2.columns

Index(['id', 'age', 'gender', 'bicb', 'caq_log', 'o_aestheticappreciation',
       'o_inquisitiveness', 'o_creativity', 'o_unconventionality', 'b_self',
       'b_r1', 'b_r2', 'b_r3', 'b_cbowu', 'b_cbows', 'b_cboww', 'b_tasa',
       'b_glov', 'r_self', 'r_r1', 'r_r2', 'r_r3', 'r_cbowu', 'r_cbows',
       'r_cboww', 'r_tasa', 'r_glov', 'study'],
      dtype='object')

In [156]:
agg2['openness']=agg2[['o_aestheticappreciation','o_inquisitiveness','o_creativity','o_unconventionality']].mean(axis=1)

In [157]:
studytag='s3'
file = studytag+'_data_agg.xlsx'
agg3=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
agg3['study']=3
agg3.columns


Index(['id', 'age', 'gender', 'br_rater1', 'br_rater2', 'br_rater3', 'br_mean',
       'br_cbowu', 'br_cbows', 'br_cboww', 'br_tasa', 'br_glov', 'gf_series',
       'gf_matrix', 'gf_letters', 'gf_papers', 'gf_ravens', 'gf_numbers',
       'n_ffi', 'e_ffi', 'o_ffi', 'a_ffi', 'c_ffi', 'meta1_r1', 'meta1_r2',
       'meta1_r3', 'meta2_r1', 'meta2_r2', 'meta2_r3', 'study'],
      dtype='object')

In [158]:
agg=pd.concat([agg[['study','id','bicb','caq_log','n_ffi','o_ffi']],
               agg2[['study','id','bicb','caq_log','openness']],
              agg3[['study','id','n_ffi','o_ffi']]])

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
agg.head()

,bicb,caq_log,id,n_ffi,o_ffi,openness,study
0,0.454545,2.862201,2001,1.937500,3.875000,NaN,1
1,0.705882,2.674149,2002,2.708333,3.687500,NaN,1
2,0.264706,1.252763,2003,2.562500,3.541667,NaN,1
3,0.000000,2.674149,2004,2.833333,3.958333,NaN,1
4,0.147059,2.525729,2005,3.291667,3.375000,NaN,1


In [159]:
df=df.merge(agg,left_on=('study','id'),right_on=('study','id'),how='left')
df.head()

,id,study,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,wordcnt,rater,bicb,caq_log,n_ffi,o_ffi,openness
0,1,2,0.935925,0.928005,0.926775,0.896130,2.250000,2.008450,3.0,1.361728,NaN,NaN,2.8125
1,1,3,0.877492,0.862785,0.862785,0.862785,1.000000,1.410000,NaN,NaN,1.67,3.00,NaN
2,2,2,0.929227,0.889161,0.825989,0.808062,1.433333,1.329606,11.0,1.612784,NaN,NaN,3.8125
3,2,3,1.005650,0.962950,0.928850,0.881350,3.500000,1.417500,NaN,NaN,2.33,2.83,NaN
4,3,2,0.980574,0.963390,0.940290,0.890422,3.298611,2.286986,12.0,1.447158,NaN,NaN,3.5000


In [160]:
print(len(df[['bicb','caq_log','SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt']].dropna()))
utilities.printNiceCorrTable(df,['bicb','caq_log'],['SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt'])

271


,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,rater,wordcnt
bicb,0.18***,0.16***,0.16***,0.13**,0.26***,0.13**
caq_log,-0.07,-0.04,-0.05,0.0,-0.09,-0.08


In [161]:
print(len(df[['openness','SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt']].dropna()))
utilities.printNiceCorrTable(df,['openness'],['SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt'])

142


,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,rater,wordcnt
openness,0.16*,0.24***,0.21**,0.2**,0.31***,0.12


In [162]:
print(len(df[['n_ffi','o_ffi','SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt']].dropna()))

utilities.printNiceCorrTable(df,['n_ffi','o_ffi'],['SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt'])

300


,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,rater,wordcnt
n_ffi,-0.06,-0.08,-0.09,-0.07,-0.01,-0.09
o_ffi,-0.07,-0.03,0.01,-0.0,0.39***,0.07


In [163]:
print(len(df[df['study']==1][['n_ffi','o_ffi','SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt']].dropna()))

utilities.printNiceCorrTable(df[df['study']==1],['n_ffi','o_ffi'],['SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt'])

167


,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,rater,wordcnt
n_ffi,-0.04,-0.04,-0.08,-0.07,-0.08,-0.1
o_ffi,0.04,0.14*,0.15**,0.15**,0.25***,0.09


In [62]:
utilities.printTriNiceCorrTable(df,['SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt'])

,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,rater
ewm_MEAN,0.74***,,,,
mad_MEAN,0.46***,0.51***,,,
wad_MEAN,0.39***,0.43***,0.77***,,
rater,0.29***,0.3***,0.46***,0.27***,
wordcnt,0.3***,0.31***,0.59***,0.24***,0.48***


In [164]:
#mean
utilities.printTriNiceCorrTable(df,['SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','rater','wordcnt'])

,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,rater
ewm_MEAN,0.89***,,,,
mad_MEAN,0.83***,0.9***,,,
wad_MEAN,0.73***,0.82***,0.95***,,
rater,0.42***,0.47***,0.52***,0.38***,
wordcnt,0.52***,0.54***,0.64***,0.44***,0.62***


# CRJ study 1 - 2

In [150]:
df=pd.read_csv(resultFolder+'CRJ_agg_meanmean.csv')
#df=pd.read_csv(resultFolder+'CRJ_agg_maxmean.csv')
df.head()

,id,study,SemDis_MEAN,ewm_MEAN,mad_MEAN,rater,wad_MEAN,wordcnt
0,2000,1,0.925317,0.916033,0.867167,1.916667,0.844633,1.666667
1,2001,1,0.955621,0.952367,0.886000,2.208333,0.867900,1.666667
2,2002,1,0.884456,0.877450,0.869425,2.343750,0.866925,1.250000
3,2003,1,0.944731,0.940286,0.903429,1.892857,0.865486,1.714286
4,2004,1,0.994607,0.969700,0.924267,2.041667,0.872867,2.500000


In [140]:
studytag='study1'
file = 'CRJ_'+studytag+'_survey.csv'
agg=pd.read_csv(sourceFolder+file,index_col=None)
agg['study']=1
agg.rename(columns={'subject':'id','d_age':'age','d_gender':'gender',
                    'ssci_creative_identity':'ssci_identity',
                   'ssci_creative_efficacy':'ssci_efficacy'},inplace=True)
agg.head()

,id,age,gender,bicb,o_bfas,i_bfas,ssci_identity,ssci_efficacy,study
0,2000,42,1,0.058823529,4.2,3.1,3,3.666666667,1
1,2001,23,1,0.454545455,3.9,4.3,3.5,4.666666667,1
2,2002,27,0,0.705882353,3.7,3.8,4.4,4.333333333,1
3,2003,24,0,0.264705882,4.6,3.8,4.6,4.333333333,1
4,2004,18,1,0,3.6,3.7,4.4,3.666666667,1


In [141]:
for c in ['bicb','ssci_identity','ssci_efficacy','o_bfas','i_bfas']:
    agg.loc[:,c]=pd.to_numeric(agg.loc[:,c],errors='coerce')

In [142]:
studytag='study2'
file = 'CRJ_'+studytag+'_survey.csv'
agg2=pd.read_csv(sourceFolder+file,index_col=None)
agg2['study']=2
agg2.rename(columns={'subject':'id','BICB_score':'bicb',
                    'Creative_Self_Efficacy':'ssci_efficacy',
                    'Creative_Personal_Identity':'ssci_identity'},inplace=True)

agg2.head()

,id,open_score,ssci_efficacy,ssci_identity,Creative_Self_Concept,bicb,gender,age,study
0,84176,3.833333,4.000000,3.8,3.909091,2.0,2.0,19.0,2
1,84177,2.916667,3.333333,3.2,3.272727,6.0,2.0,19.0,2
2,84178,4.083333,4.166667,4.4,4.272727,13.0,1.0,19.0,2
3,84188,4.166667,4.500000,4.4,4.454545,15.0,1.0,22.0,2
4,84193,3.833333,3.666667,2.4,3.090909,10.0,2.0,19.0,2


In [143]:
agg=pd.concat([agg[['study','id','bicb','ssci_efficacy','ssci_identity','o_bfas','i_bfas']],
               agg2[['study','id','bicb','ssci_efficacy','ssci_identity','open_score']]])


/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [151]:
df=df.merge(agg,left_on=('study','id'),right_on=('study','id'),how='left')
df.head()

,id,study,SemDis_MEAN,ewm_MEAN,mad_MEAN,rater,wad_MEAN,wordcnt,bicb,i_bfas,o_bfas,open_score,ssci_efficacy,ssci_identity
0,2000,1,0.925317,0.916033,0.867167,1.916667,0.844633,1.666667,0.058824,3.1,4.2,NaN,3.666667,3.0
1,2001,1,0.955621,0.952367,0.886000,2.208333,0.867900,1.666667,0.454545,4.3,3.9,NaN,4.666667,3.5
2,2002,1,0.884456,0.877450,0.869425,2.343750,0.866925,1.250000,0.705882,3.8,3.7,NaN,4.333333,4.4
3,2003,1,0.944731,0.940286,0.903429,1.892857,0.865486,1.714286,0.264706,3.8,4.6,NaN,4.333333,4.6
4,2004,1,0.994607,0.969700,0.924267,2.041667,0.872867,2.500000,0.000000,3.7,3.6,NaN,3.666667,4.4


In [147]:
print(len(df[df['study']==1][['bicb','ssci_efficacy','ssci_identity','o_bfas','i_bfas']].dropna()))
utilities.printNiceCorrTable(df[df['study']==1],['bicb','ssci_efficacy','ssci_identity','o_bfas','i_bfas'],['rater','SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','wordcnt'])

134


,rater,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,wordcnt
bicb,0.27***,0.11,0.12,0.12,0.12,0.1
ssci_efficacy,0.24***,0.17**,0.16*,0.13,0.12,0.15*
ssci_identity,0.21**,0.17**,0.2**,0.16*,0.14*,0.2**
o_bfas,0.17**,0.06,0.17**,0.17**,0.18**,0.05
i_bfas,0.22***,0.11,0.15**,0.12,0.09,0.2**


In [149]:
#max
print(len(df[df['study']==2][['bicb','ssci_efficacy','ssci_identity','open_score','SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','wordcnt']].dropna()))
utilities.printNiceCorrTable(df[df['study']==2],['bicb','ssci_efficacy','ssci_identity','open_score'],['SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','wordcnt'])

149


,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,wordcnt
bicb,0.07,0.05,0.06,0.02,0.14*
ssci_efficacy,0.14*,0.14*,0.14*,0.11,0.09
ssci_identity,0.13,0.11,0.16*,0.16*,0.08
open_score,0.09,0.13,0.22***,0.22***,0.13


In [152]:
#mean
print(len(df[df['study']==2][['bicb','ssci_efficacy','ssci_identity','open_score','SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','wordcnt']].dropna()))
utilities.printNiceCorrTable(df[df['study']==2],['bicb','ssci_efficacy','ssci_identity','open_score'],['SemDis_MEAN','ewm_MEAN','mad_MEAN','wad_MEAN','wordcnt'])

149


,SemDis_MEAN,ewm_MEAN,mad_MEAN,wad_MEAN,wordcnt
bicb,0.02,-0.0,-0.01,-0.04,0.08
ssci_efficacy,0.05,0.05,0.03,-0.01,0.05
ssci_identity,0.07,0.05,0.09,0.07,0.04
open_score,0.02,0.05,0.14*,0.16*,0.05


# Study 1

In [193]:
studytag='s1'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
file = studytag+'_yy_agg.csv'
df=pd.read_csv(resultFolder+file)
agg.columns

Index(['id', 'age', 'gender', 'b_rater1', 'b_rater2', 'b_rater3', 'b_rater4',
       'b_cbowu', 'b_cbows', 'b_cboww', 'b_tasa', 'b_glov', 'r_rater1',
       'r_rater2', 'r_rater3', 'r_rater4', 'r_cbowu', 'r_cbows', 'r_cboww',
       'r_tasa', 'r_glov', 'b_cbowu_a', 'b_cbows_a', 'b_cboww_a', 'b_tasa_a',
       'b_glov_a', 'r_cbowu_a', 'r_cbows_a', 'r_cboww_a', 'r_tasa_a',
       'r_glov_a', 'gf_cfiq', 'gf_letters', 'gf_numbers', 'bicb', 'caq_log',
       'icaa_act', 'icaa_ach', 'ssci_creative_identity',
       'ssci_creative_efficacy', 'n_ffi', 'e_ffi', 'o_ffi', 'a_ffi', 'c_ffi',
       'meta1_r1', 'meta1_r2', 'meta1_r3', 'meta1_r4', 'meta2_r1', 'meta2_r2',
       'meta2_r3', 'meta2_r4'],
      dtype='object')

In [116]:
suffixList=['b_','r_']
sdSpace=['cbowu','cbows','cboww','tasa','glov']

col1=['bicb','o_ffi','ssci_creative_efficacy','ssci_creative_identity','icaa_ach','caq_log','gf_letters','gf_numbers']
allDF=agg[col1]
allDF['pc_meta']=getPC(agg,['meta1_r1', 'meta1_r2', 'meta1_r3', 'meta1_r4', 'meta2_r1', 'meta2_r2',\
       'meta2_r3', 'meta2_r4'])
allDF['m_meta']=agg[['meta1_r1', 'meta1_r2', 'meta1_r3', 'meta1_r4', 'meta2_r1', 'meta2_r2',\
       'meta2_r3', 'meta2_r4']].mean(axis=1)
if allDF[['pc_meta','m_meta']].corr().values[0,1]<0:
        allDF['pc_meta']=-allDF['pc_meta']
col1=col1+['pc_meta','m_meta']

for s in suffixList:
    raterCol=[s+'rater1',s+'rater2',s+'rater3',s+'rater4']
    allDF[s+'rater']=agg[raterCol].mean(axis=1)
allDF['rater']=allDF[[s+'rater' for s in suffixList]].mean(axis=1)

for s in suffixList:
    allDF['pc_sd_'+s]=getPC(agg,[s+c for c in sdSpace])
    allDF['m_sd_'+s]=agg[[s+c for c in sdSpace]].mean(axis=1)
    if allDF[['pc_sd_'+s,'m_sd_'+s]].corr().values[0,1]<0:
        allDF['pc_sd_'+s]=-allDF['pc_sd_'+s]
allDF['pc_sd']=allDF[['pc_sd_'+s for s in suffixList]].mean(axis=1)
allDF['m_sd']=allDF[['m_sd_'+s for s in suffixList]].mean(axis=1)
col2=['pc_sd','m_sd']

algomethod=['min_','orig_','disp_','volume_']
for s in suffixList:
    for m in algomethod:
        allDF['pc_'+m+s]=getPC(df,[s+m+p for p in splist])
        allDF['m_'+m+s]=df[[s+m+p for p in splist]].mean(axis=1)
        if allDF[['pc_'+m+s,'m_'+m+s]].corr().values[0,1]<0:
            allDF['pc_'+m+s]=-allDF['pc_'+m+s]
for m in algomethod:
    allDF['pc_'+m]=allDF[['pc_'+m+s for s in suffixList]].mean(axis=1)
    allDF['m_'+m]=allDF[['m_'+m+s for s in suffixList]].mean(axis=1)
    allDF['fluency']=df[[s+'fluency' for s in suffixList]].mean(axis=1)
col3=['fluency']
for m in algomethod:
    col3=col3+['pc_'+m,'m_'+m]
    

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [161]:

printNiceCorrTable(allDF,col3,col3)

,fluency,pc_min_,m_min_,pc_orig_,m_orig_,pc_disp_,m_disp_,pc_volume_,m_volume_
fluency,1.0,-0.38***,-0.35***,0.03,0.14*,0.73***,0.71***,0.85***,0.87***
pc_min_,-0.38***,1.0,0.99***,0.55***,0.4***,-0.16**,-0.14*,-0.46***,-0.47***
m_min_,-0.35***,0.99***,1.0,0.55***,0.4***,-0.13*,-0.12,-0.45***,-0.44***
pc_orig_,0.03,0.55***,0.55***,1.0,0.96***,0.18**,0.18**,-0.09,-0.08
m_orig_,0.14*,0.4***,0.4***,0.96***,1.0,0.27***,0.26***,0.02,0.03
pc_disp_,0.73***,-0.16**,-0.13*,0.18**,0.27***,1.0,0.99***,0.41***,0.48***
m_disp_,0.71***,-0.14*,-0.12,0.18**,0.26***,0.99***,1.0,0.39***,0.46***
pc_volume_,0.85***,-0.46***,-0.45***,-0.09,0.02,0.41***,0.39***,1.0,0.96***
m_volume_,0.87***,-0.47***,-0.44***,-0.08,0.03,0.48***,0.46***,0.96***,1.0


In [160]:
printNiceCorrTable(allDF,col1+['rater'],col2)

,pc_sd,m_sd
bicb,0.11,0.12
o_ffi,0.03,0.05
ssci_creative_efficacy,0.18**,0.19**
ssci_creative_identity,0.24***,0.25***
icaa_ach,0.06,0.07
caq_log,0.03,0.05
gf_letters,0.02,0.01
gf_numbers,0.02,0.01
pc_meta,0.22***,0.21***
m_meta,0.23***,0.22***


In [159]:
printNiceCorrTable(allDF,col1+['rater'],col3)

,fluency,pc_min_,m_min_,pc_orig_,m_orig_,pc_disp_,m_disp_,pc_volume_,m_volume_
bicb,0.07,0.22***,0.26***,0.16*,0.15*,0.14*,0.14*,-0.01,-0.01
o_ffi,0.02,0.12,0.15**,0.08,0.09,0.01,0.01,-0.01,-0.01
ssci_creative_efficacy,0.1,0.21**,0.24***,0.22***,0.21**,0.12,0.12,0.05,0.07
ssci_creative_identity,0.03,0.23***,0.25***,0.24***,0.23***,0.09,0.08,0.03,0.03
icaa_ach,0.19**,0.17**,0.18**,0.21**,0.2**,0.15*,0.16*,0.14*,0.13
caq_log,0.07,0.1,0.14*,0.05,0.07,0.08,0.09,0.06,0.07
gf_letters,-0.15*,0.1,0.1,0.05,0.03,-0.06,-0.05,-0.2**,-0.17**
gf_numbers,-0.03,0.14*,0.14*,0.04,0.0,0.0,0.01,-0.07,-0.07
pc_meta,-0.09,0.27***,0.27***,0.09,0.05,-0.03,-0.02,-0.12,-0.11
m_meta,-0.1,0.28***,0.28***,0.09,0.05,-0.03,-0.02,-0.13,-0.12


In [158]:
linear_regressor = LinearRegression()  # create object for the class
iv='fluency'
col4=[]
for c in col3:
    if not c==iv:
        tmp=allDF[[iv,c]].dropna()
        linear_regressor.fit(tmp[[iv]].values, tmp[c].values)  # perform linear regression
        allDF['res_'+c]=allDF[c].values-linear_regressor.predict(allDF[[iv]].values)  # make predictions
        col4=col4+['res_'+c]

printNiceCorrTable(allDF,col1+['rater'],col4)

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,res_pc_min_,res_m_min_,res_pc_orig_,res_m_orig_,res_pc_disp_,res_m_disp_,res_pc_volume_,res_m_volume_
bicb,0.27***,0.3***,0.16*,0.14*,0.13,0.13,-0.14,-0.16*
o_ffi,0.13*,0.17**,0.08,0.08,-0.01,-0.0,-0.05,-0.07
ssci_creative_efficacy,0.27***,0.29***,0.22***,0.2**,0.08,0.07,-0.06,-0.04
ssci_creative_identity,0.26***,0.27***,0.24***,0.23***,0.09,0.08,-0.0,-0.01
icaa_ach,0.26***,0.27***,0.21**,0.18**,0.02,0.04,-0.03,-0.09
caq_log,0.13*,0.18**,0.05,0.06,0.05,0.06,-0.01,-0.0
gf_letters,0.05,0.05,0.06,0.05,0.07,0.08,-0.14*,-0.08
gf_numbers,0.13*,0.13*,0.04,0.01,0.04,0.05,-0.08,-0.09
pc_meta,0.26***,0.25***,0.09,0.06,0.06,0.07,-0.07,-0.07
m_meta,0.27***,0.26***,0.1,0.07,0.06,0.07,-0.08,-0.08


In [27]:
suffixList=['b_','r_']
sdSpace=['cbowu','cbows','cboww','tasa','glov']

col1=['bicb','o_ffi','ssci_creative_efficacy','ssci_creative_identity','icaa_ach','caq_log','gf_letters','gf_numbers']
allDF=agg[col1]
allDF['meta']=agg[['meta1_r1', 'meta1_r2', 'meta1_r3', 'meta1_r4', 'meta2_r1', 'meta2_r2',
       'meta2_r3', 'meta2_r4']].mean(axis=1)
col1=col1+['meta']
col2=sdSpace
for c in col2:
    allDF[c]=agg[[s+c for s in suffixList]].mean(axis=1)
col3=['min_'+p for p in splist]+['orig_'+p for p in splist]+\
            ['disp_'+p for p in splist]+['volume_'+p for p in splist]+['fluency']
for c in col3:
    allDF[c]=df[[s+c for s in suffixList]].mean(axis=1)
    
for s in suffixList:
    raterCol=[s+'rater1',s+'rater2',s+'rater3',s+'rater4']
    allDF[s+'rater']=agg[raterCol].mean(axis=1)
allDF['rater']=allDF[[s+'rater' for s in suffixList]].mean(axis=1)
allDF.rename(columns=dict([(c,renameLabel(c)) for c in col3]),inplace=True)
col3=[renameLabel(c) for c in col3]

display(np.round(allDF[col3].corr(),2).style.set_table_styles([dict(selector="th",props=[('max-width', '75px')])]))
display(np.round(allDF.corr().loc[col1+['rater'],col2],2))
display(np.round(allDF.corr().loc[col1+['rater'],col3],2))

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,min_1,min_2,min_3,min_4,orig_1,orig_2,orig_3,orig_4,disp_1,disp_2,disp_3,disp_4,volume_1,volume_2,volume_3,volume_4,fluency
min_1,1,0.88,0.89,0.8,0.26,0.67,-0.3,0.63,-0.07,-0.15,0.07,-0.29,-0.4,-0.41,-0.36,-0.38,-0.33
min_2,0.88,1,0.82,0.81,0.17,0.63,-0.37,0.55,-0.09,-0.14,0.07,-0.3,-0.46,-0.44,-0.39,-0.36,-0.37
min_3,0.89,0.82,1,0.77,0.24,0.66,-0.3,0.6,-0.02,-0.09,0.14,-0.23,-0.39,-0.4,-0.33,-0.4,-0.29
min_4,0.8,0.81,0.77,1,0.12,0.5,-0.27,0.42,-0.12,-0.19,-0.01,-0.3,-0.43,-0.43,-0.36,-0.38,-0.34
orig_1,0.26,0.17,0.24,0.12,1,0.44,0.21,0.51,0.28,0.28,0.24,0.25,0.05,0.06,0.05,-0.02,0.16
orig_2,0.67,0.63,0.66,0.5,0.44,1,-0.13,0.83,0.01,-0.02,0.14,-0.14,-0.21,-0.21,-0.18,-0.26,-0.15
orig_3,-0.3,-0.37,-0.3,-0.27,0.21,-0.13,1,-0.08,0.33,0.35,0.07,0.44,0.39,0.4,0.35,0.3,0.38
orig_4,0.63,0.55,0.6,0.42,0.51,0.83,-0.08,1,0.14,0.1,0.22,-0.01,-0.1,-0.12,-0.08,-0.18,-0.03
disp_1,-0.07,-0.09,-0.02,-0.12,0.28,0.01,0.33,0.14,1,0.97,0.77,0.93,0.41,0.38,0.51,0.21,0.68
disp_2,-0.15,-0.14,-0.09,-0.19,0.28,-0.02,0.35,0.1,0.97,1,0.76,0.96,0.46,0.44,0.56,0.28,0.71


,cbowu,cbows,cboww,tasa,glov
bicb,0.12,0.10,0.11,0.17,0.07
o_ffi,0.07,-0.03,0.01,0.14,0.04
ssci_creative_efficacy,0.20,0.11,0.17,0.26,0.17
ssci_creative_identity,0.26,0.21,0.23,0.31,0.20
icaa_ach,0.10,0.01,0.06,0.18,0.02
caq_log,0.07,0.04,0.03,0.17,-0.01
gf_letters,0.01,0.06,-0.02,0.02,-0.02
gf_numbers,0.03,0.02,0.02,-0.01,-0.01
meta,0.22,0.24,0.21,0.28,0.14
rater,0.60,0.56,0.54,0.51,0.45


,min_1,min_2,min_3,min_4,orig_1,orig_2,orig_3,orig_4,disp_1,disp_2,disp_3,disp_4,volume_1,volume_2,volume_3,volume_4,fluency
bicb,0.24,0.23,0.29,0.19,0.13,0.15,0.02,0.13,0.14,0.14,0.15,0.11,-0.02,-0.02,0.04,-0.05,0.07
o_ffi,0.15,0.11,0.13,0.17,0.05,0.12,0.04,0.03,0.03,0.03,-0.02,0.00,-0.01,-0.00,0.01,-0.06,0.02
ssci_creative_efficacy,0.24,0.18,0.27,0.19,0.11,0.22,0.05,0.21,0.12,0.12,0.10,0.11,0.07,0.06,0.08,0.06,0.10
ssci_creative_identity,0.25,0.20,0.26,0.20,0.10,0.29,0.06,0.21,0.08,0.09,0.06,0.07,0.02,0.02,0.04,-0.01,0.03
icaa_ach,0.17,0.14,0.19,0.19,0.16,0.18,0.03,0.20,0.15,0.15,0.17,0.13,0.14,0.14,0.18,0.02,0.19
caq_log,0.13,0.09,0.15,0.14,0.01,0.04,0.09,0.05,0.11,0.11,0.05,0.08,0.07,0.07,0.10,0.05,0.07
gf_letters,0.09,0.10,0.10,0.08,0.12,0.05,-0.05,-0.03,-0.03,-0.04,-0.03,-0.10,-0.16,-0.15,-0.17,-0.17,-0.15
gf_numbers,0.13,0.09,0.18,0.11,0.06,0.00,-0.10,0.05,0.04,0.01,0.05,-0.05,-0.04,-0.07,-0.04,-0.09,-0.03
meta,0.27,0.24,0.24,0.31,0.02,0.16,-0.10,0.07,0.01,-0.03,0.02,-0.08,-0.10,-0.14,-0.11,-0.13,-0.10
rater,0.74,0.71,0.73,0.68,0.26,0.53,-0.28,0.47,-0.03,-0.10,0.05,-0.21,-0.35,-0.35,-0.30,-0.35,-0.27


In [33]:
linear_regressor = LinearRegression()  # create object for the class
iv='fluency'
col4=[]
for c in col3:
    if not c==iv:
        tmp=allDF[[iv,c]].dropna()
        linear_regressor.fit(tmp[[iv]].values, tmp[c].values)  # perform linear regression
        allDF['res_'+c]=allDF[c].values-linear_regressor.predict(allDF[[iv]].values)  # make predictions
        col4=col4+['res_'+c]

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [34]:
display(np.round(allDF.corr().loc[col1+['rater'],col4],2))

,res_min_1,res_min_2,res_min_3,res_min_4,res_orig_1,res_orig_2,res_orig_3,res_orig_4,res_disp_1,res_disp_2,res_disp_3,res_disp_4,res_volume_1,res_volume_2,res_volume_3,res_volume_4
bicb,0.27,0.27,0.33,0.23,0.12,0.16,-0.01,0.13,0.12,0.13,0.13,0.08,-0.20,-0.17,-0.09,-0.12
o_ffi,0.16,0.13,0.15,0.19,0.04,0.12,0.04,0.03,0.02,0.03,-0.03,-0.02,-0.07,-0.06,-0.04,-0.09
ssci_creative_efficacy,0.28,0.23,0.31,0.24,0.10,0.24,0.01,0.21,0.08,0.07,0.06,0.05,-0.05,-0.02,-0.06,0.00
ssci_creative_identity,0.28,0.22,0.28,0.23,0.10,0.29,0.05,0.21,0.08,0.09,0.05,0.07,-0.02,-0.01,0.01,-0.04
icaa_ach,0.24,0.23,0.26,0.27,0.13,0.21,-0.05,0.20,0.03,0.01,0.08,-0.02,-0.08,-0.07,-0.03,-0.13
caq_log,0.17,0.13,0.17,0.18,-0.00,0.05,0.06,0.05,0.08,0.08,0.01,0.04,-0.01,-0.00,0.06,-0.00
gf_letters,0.04,0.04,0.06,0.03,0.14,0.03,0.01,-0.03,0.10,0.10,0.05,0.04,-0.05,-0.04,-0.09,-0.09
gf_numbers,0.12,0.09,0.18,0.10,0.07,-0.00,-0.09,0.05,0.08,0.04,0.08,-0.04,-0.03,-0.08,-0.05,-0.09
meta,0.25,0.22,0.22,0.30,0.03,0.15,-0.06,0.07,0.11,0.06,0.08,-0.00,-0.03,-0.09,-0.06,-0.09
rater,0.69,0.66,0.69,0.63,0.30,0.50,-0.19,0.47,0.21,0.14,0.22,0.00,-0.25,-0.24,-0.16,-0.25


# Study 2

In [162]:
studytag='s2'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
file = studytag+'_yy_agg.csv'
df=pd.read_csv(resultFolder+file)
agg.columns

Index(['id', 'age', 'gender', 'bicb', 'caq_log', 'o_aestheticappreciation',
       'o_inquisitiveness', 'o_creativity', 'o_unconventionality', 'b_self',
       'b_r1', 'b_r2', 'b_r3', 'b_cbowu', 'b_cbows', 'b_cboww', 'b_tasa',
       'b_glov', 'r_self', 'r_r1', 'r_r2', 'r_r3', 'r_cbowu', 'r_cbows',
       'r_cboww', 'r_tasa', 'r_glov'],
      dtype='object')

In [163]:
suffixList=['b_','r_']
sdSpace=['cbowu','cbows','cboww','tasa','glov']

col1=['bicb', 'caq_log', 'o_aestheticappreciation','o_inquisitiveness', 'o_creativity', 'o_unconventionality', 'b_self',]
allDF=agg[col1]


for s in suffixList:
    raterCol=[s+'r1',s+'r2',s+'r3']
    allDF[s+'rater']=agg[raterCol].mean(axis=1)
allDF['rater']=allDF[[s+'rater' for s in suffixList]].mean(axis=1)

for s in suffixList:
    allDF['pc_sd_'+s]=getPC(agg,[s+c for c in sdSpace])
    allDF['m_sd_'+s]=agg[[s+c for c in sdSpace]].mean(axis=1)
    if allDF[['pc_sd_'+s,'m_sd_'+s]].corr().values[0,1]<0:
        allDF['pc_sd_'+s]=-allDF['pc_sd_'+s]
allDF['pc_sd']=allDF[['pc_sd_'+s for s in suffixList]].mean(axis=1)
allDF['m_sd']=allDF[['m_sd_'+s for s in suffixList]].mean(axis=1)
col2=['pc_sd','m_sd']

algomethod=['min_','orig_','disp_','volume_']
for s in suffixList:
    for m in algomethod:
        allDF['pc_'+m+s]=getPC(df,[s+m+p for p in splist])
        allDF['m_'+m+s]=df[[s+m+p for p in splist]].mean(axis=1)
        if allDF[['pc_'+m+s,'m_'+m+s]].corr().values[0,1]<0:
            allDF['pc_'+m+s]=-allDF['pc_'+m+s]
for m in algomethod:
    allDF['pc_'+m]=allDF[['pc_'+m+s for s in suffixList]].mean(axis=1)
    allDF['m_'+m]=allDF[['m_'+m+s for s in suffixList]].mean(axis=1)
    allDF['fluency']=df[[s+'fluency' for s in suffixList]].mean(axis=1)
col3=['fluency']
for m in algomethod:
    col3=col3+['pc_'+m,'m_'+m]

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [164]:
printNiceCorrTable(allDF,col3,col3)

printNiceCorrTable(allDF,col1+['rater'],col2)


printNiceCorrTable(allDF,col1+['rater'],col3)

,fluency,pc_min_,m_min_,pc_orig_,m_orig_,pc_disp_,m_disp_,pc_volume_,m_volume_
fluency,1.0,-0.46***,-0.45***,-0.11,0.19**,0.74***,0.73***,0.81***,0.87***
pc_min_,-0.46***,1.0,0.99***,0.74***,0.42***,-0.38***,-0.36***,-0.46***,-0.51***
m_min_,-0.45***,0.99***,1.0,0.73***,0.41***,-0.36***,-0.35***,-0.45***,-0.49***
pc_orig_,-0.11,0.74***,0.73***,1.0,0.79***,-0.11,-0.1,-0.17**,-0.2**
m_orig_,0.19**,0.42***,0.41***,0.79***,1.0,0.12,0.13,0.13,0.13
pc_disp_,0.74***,-0.38***,-0.36***,-0.11,0.12,1.0,1.0***,0.39***,0.52***
m_disp_,0.73***,-0.36***,-0.35***,-0.1,0.13,1.0***,1.0,0.38***,0.51***
pc_volume_,0.81***,-0.46***,-0.45***,-0.17**,0.13,0.39***,0.38***,1.0,0.96***
m_volume_,0.87***,-0.51***,-0.49***,-0.2**,0.13,0.52***,0.51***,0.96***,1.0


,pc_sd,m_sd
bicb,0.05,0.05
caq_log,-0.01,-0.0
o_aestheticappreciation,0.17**,0.16*
o_inquisitiveness,0.11,0.08
o_creativity,0.13,0.14*
o_unconventionality,0.07,0.05
b_self,0.07,0.08
rater,0.5***,0.51***


,fluency,pc_min_,m_min_,pc_orig_,m_orig_,pc_disp_,m_disp_,pc_volume_,m_volume_
bicb,0.02,0.03,0.03,-0.02,0.07,0.09,0.08,-0.0,-0.03
caq_log,0.05,0.01,0.02,-0.02,0.04,0.11,0.11,0.01,-0.01
o_aestheticappreciation,-0.18**,0.25***,0.25***,0.13,0.15*,-0.1,-0.1,-0.2**,-0.21**
o_inquisitiveness,-0.17**,0.14,0.12,0.02,0.01,-0.06,-0.06,-0.2**,-0.23***
o_creativity,-0.03,0.14*,0.15*,0.02,0.1,0.03,0.02,-0.06,-0.08
o_unconventionality,-0.11,0.09,0.09,0.0,0.03,-0.04,-0.04,-0.14*,-0.15*
b_self,0.14,0.1,0.1,0.09,0.12,0.12,0.12,0.07,0.07
rater,-0.38***,0.72***,0.72***,0.48***,0.27***,-0.29***,-0.28***,-0.36***,-0.41***


In [165]:
linear_regressor = LinearRegression()  # create object for the class
iv='fluency'
col4=[]
for c in col3:
    if not c==iv:
        tmp=allDF[[iv,c]].dropna()
        linear_regressor.fit(tmp[[iv]].values, tmp[c].values)  # perform linear regression
        allDF['res_'+c]=allDF[c].values-linear_regressor.predict(allDF[[iv]].values)  # make predictions
        col4=col4+['res_'+c]
printNiceCorrTable(allDF,col1+['rater'],col4)

,res_pc_min_,res_m_min_,res_pc_orig_,res_m_orig_,res_pc_disp_,res_m_disp_,res_pc_volume_,res_m_volume_
bicb,0.04,0.05,-0.02,0.07,0.1,0.1,-0.03,-0.08
caq_log,0.04,0.05,-0.02,0.03,0.12,0.12,-0.04,-0.07
o_aestheticappreciation,0.18**,0.19**,0.11,0.19**,0.04,0.04,-0.09,-0.1
o_inquisitiveness,0.07,0.05,0.0,0.04,0.09,0.08,-0.11,-0.15*
o_creativity,0.15*,0.15*,0.02,0.11,0.07,0.06,-0.07,-0.13
o_unconventionality,0.05,0.05,-0.01,0.05,0.07,0.06,-0.09,-0.09
b_self,0.18**,0.17**,0.11,0.1,0.02,0.02,-0.08,-0.14*
rater,0.61***,0.61***,0.44***,0.35***,-0.02,-0.01,-0.1,-0.19**


In [100]:
suffixList=['b_','r_']
sdSpace=['cbowu','cbows','cboww','tasa','glov']

col1=['bicb', 'caq_log', 'o_aestheticappreciation','o_inquisitiveness', 'o_creativity', 'o_unconventionality', 'b_self',]
allDF=agg[col1]

col2=sdSpace
for c in col2:
    allDF[c]=agg[[s+c for s in suffixList]].mean(axis=1)
col3=['min_'+p for p in splist]+['orig_'+p for p in splist]+\
            ['disp_'+p for p in splist]+['volume_'+p for p in splist]+['fluency']
for c in col3:
    allDF[c]=df[[s+c for s in suffixList]].mean(axis=1)
    
for s in suffixList:
    raterCol=[s+'r1',s+'r2',s+'r3']
    allDF[s+'rater']=agg[raterCol].mean(axis=1)
allDF['rater']=allDF[[s+'rater' for s in suffixList]].mean(axis=1)
allDF.rename(columns=dict([(c,renameLabel(c)) for c in col3]),inplace=True)
col3=[renameLabel(c) for c in col3]

display(np.round(allDF[col3].corr(),2).style.set_table_styles([dict(selector="th",props=[('max-width', '75px')])]))
display(np.round(allDF.corr().loc[col1+['rater'],col2],2))
display(np.round(allDF.corr().loc[col1+['rater'],col3],2))

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,min_1,min_2,min_3,min_4,orig_1,orig_2,orig_3,orig_4,disp_1,disp_2,disp_3,disp_4,volume_1,volume_2,volume_3,volume_4,fluency
min_1,1,0.92,0.91,0.84,0.24,0.59,-0.53,0.64,-0.35,-0.44,-0.09,-0.52,-0.48,-0.46,-0.39,-0.49,-0.44
min_2,0.92,1,0.85,0.85,0.1,0.61,-0.58,0.64,-0.37,-0.44,-0.11,-0.54,-0.5,-0.47,-0.4,-0.5,-0.46
min_3,0.91,0.85,1,0.79,0.23,0.6,-0.48,0.68,-0.23,-0.33,0.04,-0.42,-0.44,-0.44,-0.35,-0.49,-0.38
min_4,0.84,0.85,0.79,1,0.19,0.53,-0.43,0.57,-0.28,-0.38,0.01,-0.46,-0.44,-0.43,-0.36,-0.52,-0.41
orig_1,0.24,0.1,0.23,0.19,1,0.27,0.22,0.29,0.14,0.11,0.17,0.12,0.19,0.17,0.23,0.08,0.18
orig_2,0.59,0.61,0.6,0.53,0.27,1,-0.26,0.81,-0.05,-0.1,0.08,-0.18,-0.17,-0.17,-0.09,-0.27,-0.08
orig_3,-0.53,-0.58,-0.48,-0.43,0.22,-0.26,1,-0.38,0.36,0.41,0.08,0.46,0.47,0.46,0.44,0.35,0.44
orig_4,0.64,0.64,0.68,0.57,0.29,0.81,-0.38,1,-0.09,-0.14,0.13,-0.22,-0.14,-0.15,-0.03,-0.23,-0.07
disp_1,-0.35,-0.37,-0.23,-0.28,0.14,-0.05,0.36,-0.09,1,0.97,0.76,0.94,0.43,0.39,0.57,0.16,0.69
disp_2,-0.44,-0.44,-0.33,-0.38,0.11,-0.1,0.41,-0.14,0.97,1,0.73,0.97,0.55,0.51,0.66,0.32,0.76


,cbowu,cbows,cboww,tasa,glov
bicb,0.11,0.01,0.04,0.05,0.02
caq_log,0.03,-0.05,-0.01,-0.09,0.04
o_aestheticappreciation,0.23,0.13,0.13,0.09,0.12
o_inquisitiveness,0.09,0.10,0.11,0.13,-0.01
o_creativity,0.17,0.13,0.09,0.02,0.15
o_unconventionality,0.03,0.06,0.01,0.13,0.04
b_self,0.06,0.05,0.09,0.10,0.08
rater,0.56,0.43,0.46,0.29,0.44


,min_1,min_2,min_3,min_4,orig_1,orig_2,orig_3,orig_4,disp_1,disp_2,disp_3,disp_4,volume_1,volume_2,volume_3,volume_4,fluency
bicb,0.04,-0.01,0.05,0.04,0.13,-0.05,0.11,-0.01,0.11,0.07,0.05,0.08,-0.04,-0.04,-0.01,-0.06,0.02
caq_log,0.03,-0.05,0.01,0.07,0.12,-0.05,0.10,-0.04,0.12,0.10,0.12,0.10,-0.00,0.01,0.00,-0.05,0.05
o_aestheticappreciation,0.24,0.18,0.26,0.25,0.18,0.08,0.00,0.12,-0.06,-0.11,-0.07,-0.14,-0.18,-0.18,-0.20,-0.21,-0.18
o_inquisitiveness,0.11,0.11,0.15,0.08,0.11,-0.01,-0.07,0.01,-0.03,-0.06,-0.06,-0.09,-0.21,-0.20,-0.21,-0.23,-0.17
o_creativity,0.13,0.09,0.18,0.14,0.15,0.02,0.10,-0.01,0.06,0.02,0.01,0.01,-0.07,-0.09,-0.05,-0.10,-0.03
o_unconventionality,0.06,0.08,0.10,0.10,0.10,-0.06,0.04,0.01,0.01,-0.04,-0.05,-0.05,-0.10,-0.12,-0.13,-0.17,-0.11
b_self,0.09,0.06,0.11,0.10,0.10,0.04,0.02,0.14,0.14,0.10,0.11,0.09,0.04,0.05,0.12,0.01,0.14
rater,0.68,0.64,0.73,0.67,0.23,0.33,-0.36,0.46,-0.26,-0.35,-0.03,-0.41,-0.40,-0.40,-0.36,-0.41,-0.38


In [38]:
linear_regressor = LinearRegression()  # create object for the class
iv='fluency'
col4=[]
for c in col3:
    if not c==iv:
        tmp=allDF[[iv,c]].dropna()
        linear_regressor.fit(tmp[[iv]].values, tmp[c].values)  # perform linear regression
        allDF['res_'+c]=allDF[c].values-linear_regressor.predict(allDF[[iv]].values)  # make predictions
        col4=col4+['res_'+c]

In [39]:
display(np.round(allDF.corr().loc[col1+['rater'],col4],2))

,res_min_1,res_min_2,res_min_3,res_min_4,res_orig_1,res_orig_2,res_orig_3,res_orig_4,res_disp_1,res_disp_2,res_disp_3,res_disp_4,res_volume_1,res_volume_2,res_volume_3,res_volume_4
bicb,0.06,0.00,0.07,0.05,0.13,-0.04,0.12,-0.01,0.13,0.09,0.05,0.10,-0.08,-0.07,-0.05,-0.08
caq_log,0.06,-0.03,0.04,0.10,0.11,-0.04,0.08,-0.04,0.11,0.10,0.11,0.10,-0.03,-0.00,-0.09,-0.09
o_aestheticappreciation,0.18,0.11,0.21,0.20,0.21,0.07,0.09,0.11,0.09,0.04,0.01,-0.00,-0.03,-0.04,-0.08,-0.12
o_inquisitiveness,0.04,0.04,0.10,0.01,0.15,-0.03,0.01,-0.01,0.12,0.11,0.02,0.08,-0.13,-0.10,-0.12,-0.16
o_creativity,0.13,0.09,0.19,0.14,0.15,0.02,0.12,-0.01,0.11,0.06,0.02,0.05,-0.08,-0.12,-0.11,-0.12
o_unconventionality,0.01,0.04,0.06,0.06,0.12,-0.06,0.09,0.00,0.11,0.06,0.00,0.07,0.01,-0.04,-0.07,-0.12
b_self,0.17,0.13,0.18,0.17,0.08,0.05,-0.05,0.15,0.06,-0.01,0.05,-0.04,-0.19,-0.16,-0.11,-0.09
rater,0.57,0.52,0.63,0.56,0.30,0.30,-0.22,0.43,-0.00,-0.10,0.17,-0.19,-0.14,-0.14,-0.03,-0.26


# Study 3

In [6]:
studytag='s3'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
file = studytag+'_yy_agg.csv'
df=pd.read_csv(sourceFolder+file)
agg.columns

Index(['id', 'age', 'gender', 'br_rater1', 'br_rater2', 'br_rater3', 'br_mean',
       'br_cbowu', 'br_cbows', 'br_cboww', 'br_tasa', 'br_glov', 'gf_series',
       'gf_matrix', 'gf_letters', 'gf_papers', 'gf_ravens', 'gf_numbers',
       'n_ffi', 'e_ffi', 'e_ffi.1', 'a_ffi', 'c_ffi', 'meta1_r1', 'meta1_r2',
       'meta1_r3', 'meta2_r1', 'meta2_r2', 'meta2_r3'],
      dtype='object')

In [9]:
suffixList=['br_']
sdSpace=['cbowu','cbows','cboww','tasa','glov']
col1=['gf_series','gf_matrix', 'gf_letters', 'gf_papers', 'gf_ravens', 'gf_numbers']
for suffix in suffixList:
    col2=[suffix+p for p in sdSpace]
    col3=[suffix+'min_'+p for p in splist]+\
            [suffix+'orig_'+p for p in splist]+\
            [suffix+'disp_'+p for p in splist]+[suffix+'volume_'+p for p in splist]+[suffix+'fluency']
    raterCol=[suffix+'rater1',suffix+'rater2',suffix+'rater3']
    allDF=pd.concat([df[col3],agg[col1+col2+raterCol]],axis=1)
    allDF['rater']=allDF[raterCol].mean(axis=1)
    allDF.rename(columns=dict([(c,renameLabel(c)) for c in col3]),inplace=True)
    col3=[renameLabel(c) for c in col3]
    display(np.round(allDF[col3].corr(),2).style.set_table_styles([dict(selector="th",props=[('max-width', '75px')])]))
    display(np.round(allDF.corr().loc[col1+['rater'],col2],2))
    display(np.round(allDF.corr().loc[col1+['rater'],col3],2))

,br_min_1,br_min_2,br_min_3,br_min_4,br_orig_1,br_orig_2,br_orig_3,br_orig_4,br_disp_1,br_disp_2,br_disp_3,br_disp_4,br_volume_1,br_volume_2,br_volume_3,br_volume_4,br_fluency
br_min_1,1,0.81,0.86,0.78,-0.12,-0.04,-0.17,-0.01,0.33,0.16,0.35,0.05,-0.43,-0.43,-0.32,-0.43,-0.08
br_min_2,0.81,1,0.8,0.75,-0.12,0.11,-0.31,0,0.37,0.29,0.42,0.16,-0.43,-0.41,-0.32,-0.38,0.02
br_min_3,0.86,0.8,1,0.88,-0.15,-0.05,-0.14,-0.13,0.29,0.16,0.26,0.03,-0.43,-0.45,-0.34,-0.46,-0.04
br_min_4,0.78,0.75,0.88,1,-0.15,-0.04,-0.11,-0.12,0.17,0.07,0.22,-0.05,-0.43,-0.42,-0.37,-0.4,-0.12
br_orig_1,-0.12,-0.12,-0.15,-0.15,1,0.25,0.16,0.2,-0.01,0.07,-0.13,0.07,0.21,0.17,0.2,0.07,0.15
br_orig_2,-0.04,0.11,-0.05,-0.04,0.25,1,0.06,0.65,0.18,0.26,0.19,0.25,0.09,0.08,0.19,0.1,0.34
br_orig_3,-0.17,-0.31,-0.14,-0.11,0.16,0.06,1,-0.07,-0.15,-0.12,-0.4,-0.01,0.43,0.41,0.31,0.36,0.09
br_orig_4,-0.01,0,-0.13,-0.12,0.2,0.65,-0.07,1,0.11,0.14,0.19,0.17,0.12,0.09,0.2,0.09,0.29
br_disp_1,0.33,0.37,0.29,0.17,-0.01,0.18,-0.15,0.11,1,0.93,0.76,0.89,-0,-0.07,0.37,-0.26,0.68
br_disp_2,0.16,0.29,0.16,0.07,0.07,0.26,-0.12,0.14,0.93,1,0.71,0.95,0.12,0.07,0.49,-0.11,0.77


,br_cbowu,br_cbows,br_cboww,br_tasa,br_glov
gf_series,0.23,0.21,0.23,0.21,0.18
gf_matrix,0.10,0.03,0.06,0.03,0.04
gf_letters,0.12,0.14,0.16,0.14,0.07
gf_papers,0.11,0.13,0.08,0.12,0.10
gf_ravens,0.11,0.09,0.08,0.08,0.12
gf_numbers,0.05,0.14,0.10,0.14,0.08
rater,0.43,0.41,0.40,0.21,0.29


,br_min_1,br_min_2,br_min_3,br_min_4,br_orig_1,br_orig_2,br_orig_3,br_orig_4,br_disp_1,br_disp_2,br_disp_3,br_disp_4,br_volume_1,br_volume_2,br_volume_3,br_volume_4,br_fluency
gf_series,0.23,0.19,0.20,0.29,-0.18,-0.12,-0.08,-0.10,-0.01,-0.11,0.03,-0.11,-0.28,-0.26,-0.21,-0.27,-0.14
gf_matrix,0.00,-0.00,-0.06,0.03,-0.03,-0.12,0.02,-0.12,-0.10,-0.16,-0.07,-0.15,-0.10,-0.10,-0.13,-0.08,-0.16
gf_letters,0.09,0.07,0.07,0.12,0.03,0.15,0.11,0.02,0.17,0.10,0.08,0.07,-0.15,-0.17,-0.03,-0.17,0.10
gf_papers,0.16,0.13,0.17,0.16,-0.11,0.02,-0.01,-0.01,0.11,0.05,0.10,0.04,-0.17,-0.17,-0.11,-0.22,-0.00
gf_ravens,0.19,0.16,0.18,0.20,-0.06,0.04,0.04,-0.01,0.05,0.00,0.05,-0.05,-0.11,-0.08,-0.05,-0.17,-0.09
gf_numbers,0.11,0.18,0.10,0.15,0.01,0.17,-0.11,0.15,0.15,0.13,0.19,0.10,-0.08,-0.04,-0.08,0.05,0.09
rater,0.58,0.42,0.53,0.48,-0.02,0.24,-0.01,0.27,0.16,0.00,0.17,-0.03,-0.20,-0.21,-0.15,-0.19,-0.01


# Study 5

In [10]:
studytag='s5'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
file = studytag+'_yy_agg.csv'
df=pd.read_csv(sourceFolder+file)
agg.columns

Index(['id', 'age', 'gender', 'r1_novelty', 'r1_novelty.1', 'r3_novelty',
       'human_factor', 'cbowu', 'cbows', 'cboww', 'tasa', 'glov',
       'semdis_factor'],
      dtype='object')

In [12]:
suffixList=['']
sdSpace=['cbowu','cbows','cboww','tasa','glov']
col1=['human_factor']
for suffix in suffixList:
    col2=[suffix+p for p in sdSpace]
    col3=[suffix+'min_'+p for p in splist]+\
            [suffix+'orig_'+p for p in splist]+\
            [suffix+'disp_'+p for p in splist]+[suffix+'volume_'+p for p in splist]
    raterCol=[suffix+'r1_novelty',suffix+'r1_novelty.1',suffix+'r3_novelty']
    allDF=pd.concat([df[col3],agg[col1+col2+raterCol+['semdis_factor']]],axis=1)
    allDF['rater']=allDF[raterCol].mean(axis=1)
    allDF.rename(columns=dict([(c,renameLabel(c)) for c in col3]),inplace=True)
    col3=[renameLabel(c) for c in col3]
    display(np.round(allDF[col3].corr(),2).style.set_table_styles([dict(selector="th",props=[('max-width', '75px')])]))
    display(np.round(allDF.corr().loc[col1+['rater'],col2+['semdis_factor']],2))
    display(np.round(allDF.corr().loc[col1+['rater']+['semdis_factor'],col3],2))

,min_1,min_2,min_3,min_4,orig_1,orig_2,orig_3,orig_4,disp_1,disp_2,disp_3,disp_4,volume_1,volume_2,volume_3,volume_4
min_1,1,0.91,0.92,0.85,0.45,0.62,0.75,0.86,0.27,0.29,0.3,0.18,-0,-0.07,-0.03,0
min_2,0.91,1,0.87,0.85,0.33,0.67,0.71,0.82,0.17,0.33,0.25,0.17,-0.01,-0.05,-0.03,-0.01
min_3,0.92,0.87,1,0.81,0.31,0.57,0.69,0.77,0.22,0.29,0.33,0.13,-0.03,-0.08,0.01,0
min_4,0.85,0.85,0.81,1,0.4,0.64,0.71,0.77,0.22,0.31,0.25,0.22,-0.03,-0.03,0.01,0.02
orig_1,0.45,0.33,0.31,0.4,1,0.48,0.66,0.49,0.18,0.1,0.09,0.08,0.01,-0.04,-0.06,-0.01
orig_2,0.62,0.67,0.57,0.64,0.48,1,0.85,0.71,0.08,0.25,0.15,0.13,-0.05,-0.01,-0.04,-0.02
orig_3,0.75,0.71,0.69,0.71,0.66,0.85,1,0.79,0.19,0.25,0.24,0.14,-0.04,-0.05,-0.01,-0
orig_4,0.86,0.82,0.77,0.77,0.49,0.71,0.79,1,0.22,0.27,0.25,0.18,0.02,-0.05,-0.05,0
disp_1,0.27,0.17,0.22,0.22,0.18,0.08,0.19,0.22,1,0.62,0.63,0.52,0.08,0.13,0.02,0.01
disp_2,0.29,0.33,0.29,0.31,0.1,0.25,0.25,0.27,0.62,1,0.57,0.5,0.03,0.16,0.06,0.07


,cbowu,cbows,cboww,tasa,glov,semdis_factor
human_factor,0.85,0.83,0.78,0.78,0.83,0.88
rater,0.82,0.81,0.76,0.78,0.77,0.85


,min_1,min_2,min_3,min_4,orig_1,orig_2,orig_3,orig_4,disp_1,disp_2,disp_3,disp_4,volume_1,volume_2,volume_3,volume_4
human_factor,0.16,0.14,0.04,0.11,-0.01,-0.07,-0.13,0.02,0.08,0.09,0.08,0.15,-0.01,-0.02,0.10,0.12
rater,0.17,0.15,0.04,0.12,-0.02,-0.05,-0.11,0.01,0.05,0.10,0.07,0.17,0.01,-0.02,0.12,0.10
semdis_factor,0.13,0.17,0.05,0.12,0.03,-0.05,-0.11,0.09,0.03,0.00,0.13,0.08,-0.01,-0.02,0.27,0.11


# CRJ paper

In [ ]:
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
outputfolder='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/algos/'


In [181]:
studytag='study1'
col1=['bicb','o_bfas','i_bfas','ssci_creative_identity','ssci_creative_efficacy']


agg=pd.read_csv(sourceFolder+'CRJ_'+studytag+'_survey.csv')
agg.info()
agg = agg.replace(r'^\s*$', np.NaN, regex=True)
agg.rename(columns={'subject':'id'},inplace=True)
agg[col1]=agg[col1].astype(float)
agg.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 8 columns):
subject                   186 non-null int64
d_age                     186 non-null object
d_gender                  186 non-null object
bicb                      186 non-null object
o_bfas                    186 non-null object
i_bfas                    186 non-null object
ssci_creative_identity    186 non-null object
ssci_creative_efficacy    186 non-null object
dtypes: int64(1), object(7)
memory usage: 11.8+ KB


,id,d_age,d_gender,bicb,o_bfas,i_bfas,ssci_creative_identity,ssci_creative_efficacy
0,2000,42,1,0.058824,4.2,3.1,3.0,3.666667
1,2001,23,1,0.454545,3.9,4.3,3.5,4.666667
2,2002,27,0,0.705882,3.7,3.8,4.4,4.333333
3,2003,24,0,0.264706,4.6,3.8,4.6,4.333333
4,2004,18,1,0.000000,3.6,3.7,4.4,3.666667


In [182]:
#get rater from item agg data
tmp=pd.read_csv(sourceFolder+'CRJ_'+studytag+'.csv',sep=';')
tmp['rater']=tmp[['rater1','rater2','rater3','rater4']].mean(axis=1)
agg=agg.merge(tmp.groupby(['id']).mean()[['rater']].reset_index(),left_on='id',right_on='id')
col1=col1+['rater']

In [184]:
tmp=pd.read_csv(outputfolder+'CRJ_'+studytag+'yy_agg.csv')
aggdf=agg[['id']+col1].merge(tmp,left_on='id',right_on='id')

#col2=['ewm_Max','min_Max','ewm_Mean','min_Mean','disp','fluency']
col2=['ewm_Max','min_Max']
printNiceCorrTable(aggdf,col1,col2)

,ewm_Max,min_Max
bicb,0.11,0.12
o_bfas,0.18**,0.18**
i_bfas,0.15*,0.11
ssci_creative_identity,0.21**,0.16*
ssci_creative_efficacy,0.16*,0.13
rater,0.42***,0.44***


In [185]:
studytag='study2'

col1=['open_score','Creative_Self_Efficacy','Creative_Personal_Identity','Creative_Self_Concept','BICB_score']


agg=pd.read_csv(sourceFolder+'CRJ_'+studytag+'_survey.csv')
agg.info()
agg = agg.replace(r'^\s*$', np.NaN, regex=True)
agg.rename(columns={'subject':'id'},inplace=True)
agg[col1]=agg[col1].astype(float)
agg.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 8 columns):
subject                       151 non-null int64
open_score                    149 non-null float64
Creative_Self_Efficacy        149 non-null float64
Creative_Personal_Identity    149 non-null float64
Creative_Self_Concept         149 non-null float64
BICB_score                    149 non-null float64
gender                        149 non-null float64
age                           137 non-null float64
dtypes: float64(7), int64(1)
memory usage: 9.6 KB


,id,open_score,Creative_Self_Efficacy,Creative_Personal_Identity,Creative_Self_Concept,BICB_score,gender,age
0,84176,3.833333,4.000000,3.8,3.909091,2.0,2.0,19.0
1,84177,2.916667,3.333333,3.2,3.272727,6.0,2.0,19.0
2,84178,4.083333,4.166667,4.4,4.272727,13.0,1.0,19.0
3,84188,4.166667,4.500000,4.4,4.454545,15.0,1.0,22.0
4,84193,3.833333,3.666667,2.4,3.090909,10.0,2.0,19.0


In [186]:
tmp=pd.read_csv(outputfolder+'CRJ_'+studytag+'yy_agg.csv')
aggdf=agg[['id']+col1].merge(tmp,left_on='id',right_on='id')

col2=['ewm_Max','min_Max','ewm_Mean','min_Mean','disp','fluency']
printNiceCorrTable(aggdf,col1,col2)

,ewm_Max,min_Max,ewm_Mean,min_Mean,disp,fluency
open_score,0.13,0.22***,0.04,0.14*,0.19**,0.15*
Creative_Self_Efficacy,0.14*,0.13,0.04,0.02,0.15*,0.16*
Creative_Personal_Identity,0.11,0.15*,0.05,0.08,0.12,0.12
Creative_Self_Concept,0.13,0.15*,0.04,0.05,0.14*,0.15*
BICB_score,0.05,0.05,-0.0,-0.01,0.15*,0.12


In [187]:
printNiceCorrTable(aggdf,col2,col2)

,ewm_Max,min_Max,ewm_Mean,min_Mean,disp,fluency
ewm_Max,1.0,0.8***,0.76***,0.53***,0.42***,0.26***
min_Max,0.8***,1.0,0.61***,0.71***,0.42***,0.32***
ewm_Mean,0.76***,0.61***,1.0,0.84***,-0.2**,-0.34***
min_Mean,0.53***,0.71***,0.84***,1.0,-0.25***,-0.33***
disp,0.42***,0.42***,-0.2**,-0.25***,1.0,0.92***
fluency,0.26***,0.32***,-0.34***,-0.33***,0.92***,1.0
